# Data Access Instructions

Data are accessible via [an object store](https://cloud.google.com/learn/what-is-object-storage) system, which enables easy, scalable web access! 

## Imports


In [1]:
import fsspec
import urllib.request
from pathlib import Path
import xradar as xd
import pyart


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



## Find Data in the Bucket

It is stored on the jetstream cloud bucket, under the `/pythia/radar/erad2024` space.

In [2]:
# Set the URL and path for the cloud
URL = 'https://js2.jetstream-cloud.org:8001/'
path = f'pythia/radar/erad2024'


fs = fsspec.filesystem("s3", anon=True, client_kwargs=dict(endpoint_url=URL))

fs.glob(f"{path}/*")

['pythia/radar/erad2024/.DS_Store',
 'pythia/radar/erad2024/20240522_MeteoSwiss_ARPA_Lombardia',
 'pythia/radar/erad2024/baltrad',
 'pythia/radar/erad2024/dda_data',
 'pythia/radar/erad2024/zarr_radar']

### Find Data to Stream
We can easily stream the data with xradar

In [3]:
files = fs.glob("pythia/radar/erad2024/dda_data/*")[:1]
files

['pythia/radar/erad2024/dda_data/ARPA_Lombardia.20240522.151546_dealiased.nc']

In [4]:
local_files = [
    fsspec.open_local(
        f"simplecache::{URL}{i}", s3={"anon": True}, filecache={"cache_storage": "."}
    )
    for i in files
]

In [5]:
dt = xd.io.open_cfradial1_datatree(local_files[0])
radar = pyart.io.read_cfradial(local_files[0])

In [6]:
dt

DataTree('root', parent=None)
│   Dimensions:              (sweep: 8, frequency: 1)
│   Coordinates:
│     * frequency            (frequency) float32 4B 9.339e+09
│   Dimensions without coordinates: sweep
│   Data variables:
│       sweep_group_name     (sweep) <U9 288B 'sweep_0.0' ... 'sweep_7.0'
│       sweep_fixed_angle    (sweep) float32 32B ...
│       latitude             float64 8B ...
│       longitude            float64 8B ...
│       altitude             float64 8B ...
│       altitude_agl         float64 8B ...
│       time_coverage_start  |S32 32B ...
│       time_coverage_end    |S32 32B ...
│       volume_number        int32 4B ...
│       platform_type        |S32 32B ...
│       instrument_type      |S32 32B ...
│       primary_axis         |S32 32B ...
│   Attributes: (12/14)
│       Conventions:         CF-1.7
│       version:             CF-Radial-1.4
│       title:               VOL_A
│       institution:         
│       references:          Conversion software: Radx::SigmetRadxFile
│       source:              Sigmet IRIS software
│       ...                  ...
│       site_name:           Desio_Radar
│       scan_name:           VOL_A
│       scan_id:             0
│       platform_is_mobile:  false
│       ray_times_increase:  true
│       history:             Wed May 22 15:34:38 2024: ncap2 -Os fixed_angle  (:)...
├── DataTree('radar_parameters')
│       Dimensions:                   ()
│       Data variables:
│           radar_antenna_gain_v      float32 4B ...
│           radar_beam_width_h        float32 4B ...
│           radar_beam_width_v        float32 4B ...
│           radar_antenna_gain_h      float32 4B ...
│           radar_receiver_bandwidth  float32 4B ...
├── DataTree('radar_calibration')
│       Dimensions:                   ()
│       Data variables: (12/55)
│           time                      |S32 32B ...
│           pulse_width               timedelta64[ns] 8B ...
│           xmit_power_h              float32 4B ...
│           xmit_power_v              float32 4B ...
│           two_way_waveguide_loss_h  float32 4B ...
│           two_way_waveguide_loss_v  float32 4B ...
│           ...                        ...
│           zdr_correction            float32 4B ...
│           ldr_correction_h          float32 4B ...
│           ldr_correction_v          float32 4B ...
│           system_phidp              float32 4B ...
│           test_power_h              float32 4B ...
│           test_power_v              float32 4B ...
├── DataTree('georeferencing_correction')
├── DataTree('sweep_0')
│       Dimensions:                    (azimuth: 360, range: 416)
│       Coordinates:
│           time                       (azimuth) datetime64[ns] 3kB 2024-05-22T15:15:...
│         * range                      (range) float32 2kB 150.0 450.0 ... 1.246e+05
│         * azimuth                    (azimuth) float32 1kB 0.03845 1.022 ... 359.0
│           elevation                  (azimuth) float32 1kB ...
│           latitude                   float64 8B ...
│           longitude                  float64 8B ...
│           altitude                   float64 8B ...
│       Data variables: (12/27)
│           scan_rate                  (azimuth) float32 1kB ...
│           antenna_transition         (azimuth) float32 1kB ...
│           DBZ_TOT                    (azimuth, range) float32 599kB ...
│           DBZ                        (azimuth, range) float32 599kB ...
│           VEL                        (azimuth, range) float32 599kB ...
│           WIDTH                      (azimuth, range) float32 599kB ...
│           ...                         ...
│           nyquist_velocity           (azimuth) float32 1kB ...
│           unambiguous_range          (azimuth) float32 1kB ...
│           n_samples                  (azimuth) float64 3kB ...
│           measured_transmit_power_v  (azimuth) float32 1kB ...
│           measured_transmit_power_h  (azimuth) float32 1kB ...
│           r_ca

### Download the Data Locally
We can also use the url of the data to download the data locally if neccessary!

In [7]:
for file in files:
    urllib.request.urlretrieve(f"{URL}{file}", Path(file).name)